In [1]:
import os
from fastapi.testclient import TestClient
from unittest.mock import patch



In [2]:
os.chdir("..")
os.getcwd()

'/Users/marshallnicholas/Desktop/text2sql/text2sql_ui/backend'

In [4]:
from app.main import app

client = TestClient(app)

In [6]:
mock_sql_result = {
        "sql": "SELECT * FROM nuclear_power_plants WHERE status_id = 3",
        "method": "simple"
    }

with patch('app.routes.text2sql.simple_service.generate_sql', return_value=mock_sql_result):
        response = client.post(
            "/api/text2sql/simple",
            json={
                "query": "Show me all operational plants",
                "execute": False
            }
        )

In [8]:
response.json()

{'sql': 'SELECT * FROM nuclear_power_plants WHERE status_id = 3',
 'method': 'simple',
 'session_id': None,
 'resolved_query': None,
 'context_used': None,
 'iterations': None,
 'tool_calls': None,
 'execution_result': None,
 'validation_result': None}

In [5]:
test_response = client.post(
    "/api/text2sql/simple",
    json={
        "query": "Show me all operational plants",
        "execute": False
    }
)

In [6]:
test_response.json()

{'detail': 'Error calling Bedrock: An error occurred (UnrecognizedClientException) when calling the InvokeModel operation: The security token included in the request is invalid.'}

In [3]:
from app.services.bedrock_client import BedrockClient
from app.config import settings

In [4]:
def test_settings_loaded():
    """Test that required settings are loaded from .env"""
    # Verify settings exist (but don't assert specific values)
    assert settings.aws_region is not None
    assert settings.aws_access_key_id is not None
    assert settings.bedrock_model_id is not None
    assert len(settings.aws_region) > 0
    assert len(settings.aws_access_key_id) > 0
    assert len(settings.bedrock_model_id) > 0

In [6]:
from app.services.bedrock_client import BedrockClient
client = BedrockClient()
result = client.invoke_model(
    prompt="Show me all operational nuclear power plants",
    system="You are an expert SQL generator for nuclear power plant databases.",
    max_tokens=4000
)

In [7]:
result

"Certainly! To show all operational nuclear power plants, we'll need to query a database that contains information about nuclear power plants. Since I don't have access to the specific database structure you're using, I'll provide a general SQL query that should work with most common database schemas. You may need to adjust the table and column names to match your specific database.\n\nHere's a SQL query that should retrieve all operational nuclear power plants:\n\n```sql\nSELECT \n    plant_name,\n    location,\n    reactor_type,\n    capacity_mw,\n    operational_since\nFROM \n    nuclear_power_plants\nWHERE \n    operational_status = 'Operational'\nORDER BY \n    capacity_mw DESC;\n```\n\nThis query does the following:\n\n1. Selects relevant columns: plant name, location, reactor type, capacity in megawatts, and the date since which the plant has been operational.\n2. Queries the `nuclear_power_plants` table (adjust this table name if needed).\n3. Filters for plants with an operatio